---
layout: post
title: Blog Post 4
---

In this blog post, I'll write a tutorial on a simple version of the *spectral clustering* algorithm for clustering data points. 

This blog post was modified from a [Jupyter Notebook](https://nbviewer.org/github/PhilChodrow/PIC16B/blob/master/HW/spectral-clustering.ipynb) written by Dr. Phil Chodrow.  
***Note***: your blog post doesn't have to contain a lot of math. It's ok for you to give explanations like "this function is an approximation of this other function according to the math in the written assignment." 

### Notation

In all the math below: 

- Boldface capital letters like $\mathbf{A}$ refer to matrices (2d arrays of numbers). 
- Boldface lowercase letters like $\mathbf{v}$ refer to vectors (1d arrays of numbers). 
- $\mathbf{A}\mathbf{B}$ refers to a matrix-matrix product (`A@B`). $\mathbf{A}\mathbf{v}$ refers to a matrix-vector product (`A@v`). 

### Comments and Docstrings

You should plan to comment all of your code. Docstrings are not required except in Part G. 

## Introduction

In this problem, we'll study *spectral clustering*. Spectral clustering is an important tool for identifying meaningful parts of data sets with complex structure. It is a technique with roots in graph theory, where the approach is used to identify communities of nodes in a graph based on the edges connecting them. The method is flexible and allows us to cluster non graph data as well. Spectral Clustering method is a growing clustering algorithm which has performed better than many traditional clustering algorithms in many cases.  
Before we start the tutorial, let's look at an example where we *don't* need spectral clustering. 

First, let's do some standard import.

In [1]:
import numpy as np
from sklearn import datasets
from matplotlib import pyplot as plt

In [ ]:
n = 200
np.random.seed(1111)
X, y = datasets.make_blobs(n_samples=n, shuffle=True, random_state=None, centers = 2, cluster_std = 2.0)
plt.scatter(X[:,0], X[:,1])

To create this example, I first ran the code in a Jupyter Notebook, and added the line `plt.savefig("Blog-Post-4-Plot1.png")` to save the result. I then moved the file `Blog-Post-4-Plot1.png` to the images/ directory of my blog. Finally, I added the line:

```
![Blog-Post-4-Plot1.png]({{ site.baseurl }}/images/Blog-Post-4-Plot1.png) 
```
immediately beneath the code block. For the rest of the plots, we will add them with the same steps.

![Blog-Post-4-Plot1.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot1.png) 

By using the `sklearn.datasets` package and `matplotlib.pyplot` package, we have successfully visualized our data in a scatter plot.

Our main task is to separate this data set into two natural "blobs." In this case with our unlabelled data, we already have two natural blobs. To achieve our task, I will implement the K-means algorithm, which has good performance on circular-ish blobs. Before we get started, let's import `KMeans`. 

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 2)
km.fit(X)

plt.scatter(X[:,0], X[:,1], c = km.predict(X))

![Blog-Post-4-Plot2.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot2.png)

Great! We have now successfully grouped our data into clusters using K-Means algorithm.

### Harder Clustering

That was all well and good, but what if our data is "shaped weird"? 

In [ ]:
np.random.seed(1234)
n = 200
X, y = datasets.make_moons(n_samples=n, shuffle=True, noise=0.05, random_state=None)
plt.scatter(X[:,0], X[:,1])

![Blog-Post-4-Plot3.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot3.png)

In this case, we can still make out two meaningful clusters in the data, but now from the plot they aren't blobs but crescents. As before, the Euclidean coordinates of the data points are contained in the matrix `X`, while the labels of each point are contained in `y`. Now we implement k-means algorithm again:

In [ ]:
km = KMeans(n_clusters = 2)
km.fit(X)
plt.scatter(X[:,0], X[:,1], c = km.predict(X))

![Blog-Post-4-Plot4.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot4.png)

Noticed that K-Means is not working very well. That is because it is designed for circular or blob-like clusters and hence won’t work as well with this cresent-shaped data. Therefore, in the following problems, I will derive and implement spectral clustering. 

## Part A

Construct the *similarity matrix* $\mathbf{A}$. $\mathbf{A}$ should be a matrix (2d `np.ndarray`) with shape `(n, n)` (recall that `n` is the number of data points). 

When constructing the similarity matrix, use a parameter `epsilon`. For this part, we start with $\epsilon$ = 0.4. 

In [25]:
epsilon = 0.4

Basically, we construct the *similarity matrix* $\mathbf{A}$ following these rules:  
(Let `X[i]` represents the coordinates of data point `i`, and `A[i, j]` represents the entry in the i-th row and j-th column of $\mathbf{A}$)  
- `A[i,j]` = 1 if d(X[i], X[j]) < $\epsilon$
- `A[i,j]` = 1 if d(X[i], X[j]) $\geq$ $\epsilon$
- `A[i,i]` = 0 The diagonal entries should all be equal to zero.

To avoid constructing this using for-loops, we will use `pairwise_distances` from sklearn. This function that computes all the pairwise distances and collects them into an appropriate matrix.

In [33]:
from sklearn.metrics.pairwise import pairwise_distances
# Function computes all the pairwise distances and collect them into an matrix
A = pairwise_distances(X, X)

Let's take a look at the matrix we got. By checking the shape, it is a square nxn matrix. In our case, n = 200.

In [15]:
A.shape

(200, 200)

In [16]:
A

array([[0.        , 1.27292462, 1.33315598, ..., 1.9812102 , 1.68337039,
        1.94073324],
       [1.27292462, 0.        , 1.46325112, ..., 1.93729167, 1.68543003,
        1.91287315],
       [1.33315598, 1.46325112, 0.        , ..., 0.64857172, 0.35035968,
        0.60860868],
       ...,
       [1.9812102 , 1.93729167, 0.64857172, ..., 0.        , 0.30070415,
        0.04219636],
       [1.68337039, 1.68543003, 0.35035968, ..., 0.30070415, 0.        ,
        0.26255757],
       [1.94073324, 1.91287315, 0.60860868, ..., 0.04219636, 0.26255757,
        0.        ]])

Noticed that the diagonal entries are already 0. Then, let's transform the other distances into 0 and 1's as we want. We could easily do it by implementing this one-line code:

In [34]:
# If dis < epsilon, A[i, j] = 1. otherwise A[i, j] = 0
A = np.where((A < epsilon), 1, 0)
# Since 0 < epsilon, all the diagonal entries becomes 1
# We will change the diagonal entries back to 0
np.fill_diagonal(A, 0)
A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 0]])

The matrix `A` now contains information about which points are near (within distance $\epsilon$) which other points. We have successfully constructed a *similarity matrix*.

## Part B
We now pose the task of clustering the data points in `X` as the task of partitioning the rows and columns of `A`. 

Let $d_i = \sum_{j = 1}^n a_{ij}$ be the $i$th row-sum of $\mathbf{A}$, which is also called the *degree* of $i$. Let $C_0$ and $C_1$ be two clusters of the data points. We assume that every data point is in either $C_0$ or $C_1$. The cluster membership as being specified by `y`. We think of `y[i]` as being the label of point `i`. So, if `y[i] = 1`, then point `i` (and therefore row $i$ of $\mathbf{A}$) is an element of cluster $C_1$.  

The *binary norm cut objective* of a matrix $\mathbf{A}$ is the function 

$$N_{\mathbf{A}}(C_0, C_1)\equiv \mathbf{cut}(C_0, C_1)\left(\frac{1}{\mathbf{vol}(C_0)} + \frac{1}{\mathbf{vol}(C_1)}\right)\;.$$

When $N_{\mathbf{A}}(C_0, C_1)$ is small, a pair of clusters $C_0$ and $C_1$ is considered to be a "good" partition of the data. To see why, let's look at the cut term and the volume term separately. 


#### B.1 The Cut Term

$\mathbf{cut}(C_0, C_1) \equiv \sum_{i \in C_0, j \in C_1} a_{ij}$ is the *cut* of the clusters $C_0$ and $C_1$.  

First, the cut term $\mathbf{cut}(C_0, C_1)$ is the number of nonzero entries in $\mathbf{A}$ that relate points in cluster $C_0$ to points in cluster $C_1$. This term is small means that points in $C_0$ shouldn't usually be very close to points in $C_1$. 

Next, we are going to write a function called `cut(A,y)` to compute the cut term. To avoid using for-loop, I have used list comprehension to construct a list of filtered `A[i, j]` and calculated the sum of the resulted list.

In [55]:
def cut(A, y):
    """
    Purpose: Compute the cut term
    Input: A: The similarity matrix, 2d np.ndarray
           y: The labels of the data,  1d np.ndarray
    output: an integer which is the cut term of the given matrix A
    """
    sum = np.sum([A[i,j] for i in np.where(y == 0)[0] for j in np.where(y == 1)[0]])
    return sum

Check our result for the true clusters `y` using the similarity matrix calculated in the previous step:

In [59]:
cut(A, y)

13

Then, we will generate a random vector of random labels of length `n`, with each label equal to either 0 or 1, and check the cut objective for the random labels. 

In [63]:
# Generate random labels
random_label = np.random.randint(0, 2, n)

# Check the cut objective
cut(A, random_label)

1146

Noticed that the cut objective for the true labels is *much* smaller than the cut objective for the random labels. This indicates that this part of the cut objective indeed favors the true clusters over the random ones. 

#### B.2 The Volume Term 

$\mathbf{vol}(C_0) \equiv \sum_{i \in C_0}d_i$, where $d_i = \sum_{j = 1}^n a_{ij}$ is the *degree* of row $i$ (the total number of all other rows related to row $i$ through $A$). The *volume* of cluster $C_0$ is a measure of the size of the cluster. 

If we choose cluster $C_0$ to be small, then $\mathbf{vol}(C_0)$ will be small and $\frac{1}{\mathbf{vol}(C_0)}$ will be large, leading to an undesirable higher objective value. 

Synthesizing, the binary normcut objective asks us to find clusters $C_0$ and $C_1$ such that:

1. There are relatively few entries of $\mathbf{A}$ that join $C_0$ and $C_1$. 
2. Neither $C_0$ and $C_1$ are too small. 

We are going to write a function called `vols(A,y)` which computes the volumes of $C_0$ and $C_1$, returning them as a tuple.

In [69]:
def vols(A, y):
    """
    Purpose: Compute the volumn term
    Input: A: The similarity matrix, 2d np.ndarray
           y: The labels of the data,  1d np.ndarray

    Output: A tuple which contains the volumes of cluster C_0 and C_1
    """
    # volume of cluster C_0
    vol_c0 = np.sum(A[y== 0])
    # volume of cluster C_1
    vol_c1 = np.sum(A[y==1])

    return (vol_c0, vol_c1)

To compute the binary normalized cut objective of similarity matrix with the clustering vector, we will write a function called `normcut(A, y)`.

In [71]:
def normcut(A, y):
    """
    Purpose: Compute the binary normalized cut objective
    Input: A: The similarity matrix, 2d np.ndarray
           y: The labels of the data,  1d np.ndarray

    Output: A float which is the binary normalized cut objective of A and y
    """
    # computing the volumes
    vol = vols(A, y)
    # computing the norm cut
    norm = cut(A, y)*((1/vol[0])+(1/vol[1]))
    return norm

Now, compare the `normcut` objective using both the true labels `y` and the fake labels we generated above.

In [72]:
# Normcut for true labels
normcut(A, y)

0.011518412331615225

In [74]:
# Normcut for random labels
normcut(A, random_label)

1.0174722557753284

Noticed that the `normcut` objective for the true labels is much smaller than the `normcut` objective for the random labels. This makes sense, as we would expect less error in non-trivially generated values with our data.

Congratulations! We have now defined a normalized cut objective which takes small values when the input clusters are (a) joined by relatively few entries in $A$ and (b) not too small.

## Part C

One approach to clustering is to try to find a cluster vector `y` such that `normcut(A,y)` is small. However, it may not be possible to find the best clustering in practical time, even for relatively small data sets. We will need to use a math trick to solve this!

We define a new vector $\mathbf{z} \in \mathbb{R}^n$ such that: 

$$
z_i = 
\begin{cases}
    \frac{1}{\mathbf{vol}(C_0)} &\quad \text{if } y_i = 0 \\ 
    -\frac{1}{\mathbf{vol}(C_1)} &\quad \text{if } y_i = 1 \\ 
\end{cases}
$$


Note that the signs of  the elements of $\mathbf{z}$ contain all the information from $\mathbf{y}$: if $i$ is in cluster $C_0$, then $y_i = 0$ and $z_i > 0$.  
Let’s write a function called transform(A, y) to construct the vector $\mathbf{z}$.

In [76]:
def transform(A, y):
    """
    Purpose: Compute the z vector
    Input: A: The similarity matrix, 2d np.ndarray
           y: The labels of the data,  1d np.ndarray
    Output: A np.ndarray
    """
    # extracting the volumes of clusters
    vol = vols(A, y)
    # computing the n-dimensional vector z
    z = np.where(y == 0, 1/vol[0], -1/vol[1])
    return z

We will use the above function to show that 

$$\mathbf{N}_{\mathbf{A}}(C_0, C_1) = \frac{\mathbf{z}^T (\mathbf{D} - \mathbf{A})\mathbf{z}}{\mathbf{z}^T\mathbf{D}\mathbf{z}}\;,$$

where $\mathbf{D}$ is the diagonal matrix with nonzero entries $d_{ii} = d_i$, and  where $d_i = \sum_{j = 1}^n a_i$ is the degree (row-sum) from before.  

In [79]:
# First compute the life hand side
LHS = normcut(A, y)
LHS

0.011518412331615225

In [80]:
# Compute the right hand side

# Get the z vector
z = transform(A, y)
# diagonal matrix
D = np.diag(np.sum(A, axis=0))

RHS = (z @ (D - A) @ z)/(z @ D @ z)
RHS

0.011518412331615088

Since the equation above is exact, but computer arithmetic is not, we will use `np.isclose()` to check if left hand side and right hand side are equal.

In [81]:
# check if they are close
np.isclose(LHS, RHS)

True

Great! Now we have verified the above equation.

Let's also check the identity $\mathbf{z}^T\mathbf{D}\mathbb{1} = 0$, where $\mathbb{1}$ is the vector of `n` ones (i.e. `np.ones(n)`). This identity effectively says that $\mathbf{z}$ should contain roughly as many positive as negative entries. Similarly, we will also use`np.isclose()` to check if left hand side and right hand side are equal.

In [82]:
# checking if identity is close to 0
np.isclose(z.T @ D @ np.ones(n), 0)

True

We have verified that the identity $\mathbf{z}^T\mathbf{D}\mathbb{1} = 0$.

## Part D

In the part C, we saw that the problem of minimizing the normcut objective is mathematically related to the problem of minimizing the function 

$R_\mathbf{A}(\mathbf{z})\equiv \frac{\mathbf{z}^T (\mathbf{D} - \mathbf{A})\mathbf{z}}{\mathbf{z}^T\mathbf{D}\mathbf{z}}$ subject to the condition $\mathbf{z}^T\mathbf{D}\mathbb{1} = 0$. 

Turns out it's actually possible to incorporate this condition into the optimization, by substituting for $\mathbf{z}$ the orthogonal complement of $\mathbf{z}$ relative to $\mathbf{D}\mathbf{1}$. In the code below, the `orth_obj` function will handle this. 

In [85]:
def orth(u, v):
    return (u @ v) / (v @ v) * v

e = np.ones(n) 

d = D @ e

def orth_obj(z):
    z_o = z - orth(z, d)
    return (z_o @ (D - A) @ z_o)/(z_o @ D @ z_o)

Then, let's use the `minimize` function from `scipy.optimize` to minimize the function `orth_obj` with respect to $\mathbf{z}$. Note that this computation might take a little while since explicit optimization can be pretty slow. We name the minimizing vector `z_min`. 

We will use the *continuous relaxation* of the normcut problem. We have originally specified that the entries of $\mathbf{z}$ should take only one of two values (back in Part C), whereas now we're allowing the entries to have *any* value. This means that we are no longer exactly optimizing the normcut objective, but rather an approximation.

The minimize function requires an “initial guess” for what we think our minimum is near, so we’ll provide it a vector of ones.

In [101]:
from scipy import optimize

# extracting the minimum vector
z_min = optimize.minimize(orth_obj, np.ones(len(z))).x

z_min

array([ 2.57168397,  2.65952192, -0.25557377, -0.11016068, -0.51454006,
       -0.31378334, -0.5681382 , -0.41030021,  2.08499242,  1.71165096,
        2.34574041, -0.29933846,  0.50699752,  2.30473925,  2.86879528,
        2.5689054 ,  2.12020462, -0.21253969, -0.41595232, -0.39363013,
        2.35417728,  1.18536707,  2.8198377 , -0.11508422, -0.71512535,
        1.88323831, -0.28756431,  2.16476716,  2.17242051, -0.25938271,
       -0.32823862,  2.6965414 ,  2.57868177,  2.41009744,  2.13811709,
        2.65994967, -0.71512535,  2.93108104,  2.73666293, -0.3509846 ,
        2.44523814, -0.21382501, -0.30296259, -1.02668978, -0.66590363,
       -0.12820942, -0.2132045 ,  2.40439311,  2.77275035,  0.87722474,
       -0.28403835, -0.24624875,  2.8198377 ,  2.28116952, -0.80542876,
       -0.49250765,  2.70916014,  1.42613162,  2.80812761, -0.16716665,
       -0.98215657, -0.47073932,  2.0333259 , -0.24653334,  2.13780167,
       -0.41077021, -0.98215656, -0.71119048, -0.11306391,  2.86

## Part E

Since by design, only the sign of `z_min[i]` actually contains information about the cluster label of data point `i`. Let's plot the original data, using one color for points such that `z_min[i] < 0` and another color for points such that `z_min[i] >= 0`. 

In [ ]:
# if z_min[i] < 0, label 1. otherwise label 0
labels = np.where(z_min < 0, 1, 0) 

# Plotting the original data
plt.scatter(X[:,0], X[:,1], c = labels)

![Blog-Post-4-Plot5.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot5.png)

Compared with the true label plot before Part A, there is a little discrepancy, but overall it is doing pretty good.

## Part F

Explicitly optimizing the orthogonal objective is  *way* too slow to be practical. If spectral clustering required that we do this each time, no one would use it. 

The Laplacian matrix is another method we can use to put together our spectral clustering algorithm. Essentially, what we are doing is finding the eigenvalues and eigenvector that will generate our labels. 

The Rayleigh-Ritz Theorem states that the minimizing $\mathbf{z}$ must be the solution with smallest eigenvalue of the generalized eigenvalue problem 

$$ (\mathbf{D} - \mathbf{A}) \mathbf{z} = \lambda \mathbf{D}\mathbf{z}\;, \quad \mathbf{z}^T\mathbf{D}\mathbb{1} = 0$$

which is equivalent to the standard eigenvalue problem 

$$ \mathbf{D}^{-1}(\mathbf{D} - \mathbf{A}) \mathbf{z} = \lambda \mathbf{z}\;, \quad \mathbf{z}^T\mathbb{1} = 0\;.$$

Well, $\mathbb{1}$ is actually the eigenvector with smallest eigenvalue of the matrix $\mathbf{D}^{-1}(\mathbf{D} - \mathbf{A})$. So, the vector $\mathbf{z}$ that we want must be the eigenvector with  the *second*-smallest eigenvalue. 

Let's construct the matrix $\mathbf{L} = \mathbf{D}^{-1}(\mathbf{D} - \mathbf{A})$, which is often called the (normalized) *Laplacian* matrix of the similarity matrix $\mathbf{A}$. Find the eigenvector corresponding to its second-smallest eigenvalue, and call it `z_eig`.

In [ ]:
# Calculate the normalized Laplacian matrix of the similarity matrix A
L = np.linalg.inv(D) @ (D-A) 

# Extracting the eigenvalues and eigenvectors of L
eigenValues, eigenVectors = np.linalg.eig(L)

idx = eigenValues.argsort()   
sorted_values = eigenValues[idx]
sorted_vectors = eigenVectors[:,idx]
# vector corresponding to 2nd smallest eigenvector
z_eig = sorted_vectors[:, 1]
label= np.where(z_eig < 0, "blue", "red")
plt.scatter(X[:,0], X[:,1], c = label)

![Blog-Post-4-Plot6.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot6.png)

This resulting plot is very similar to the plot with original label.

## Part G

Now, let's put together everything we have done in the previous parts. Let's write a function called `spectral_clustering(X, epsilon)` which performs spectral clustering. 

In [117]:
def spectral_clustering(X, epsilon):
    """
    Purpose: Performs spectral clustering
    Input: X : The array of input data. np.ndarray
           epsilon : The distance threshold. float
    Output: Returns an array of binary labels indicating whether data point i is in group 0 or group 1
    """
    # Construct the similarity matrix A
    A = pairwise_distances(X, X) 
    A = np.where((A < epsilon), 1, 0)
    np.fill_diagonal(A, 0)
    
    # Construct the Laplacian matrix L
    D = np.diag(A.sum(axis=0)) # construct diagonal matrix
    L = np.linalg.inv(D) @ (D - A)
    
    # Compute the eigenvector with second-smallest eigenvalue of the Laplacian matrix
    evals, evecs = np.linalg.eig(L)
    sorted_vecs = evecs[:, np.argsort(evals)]
    z_eig = sorted_vecs[:, 1]
    
    # Return labels based on z_eig
    labels = np.where(z_eig < 0, 1, 0) 
    
    return labels

Let's test our function.

In [ ]:
# Generate the new labels
label = spectral_clustering(X, 0.4)
# Plotting the data
plt.scatter(X[:,0], X[:,1], c = label)

![Blog-Post-4-Plot7.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot7.png)

The plot we generate is the same as Part F. Therefore our function is able to perform the spectral clustering.

## Part H

Next, we are going to run a few experiments using our function, by generating different data sets using `make_moons`. We will see what happens when we increase the `noise`. For all the experiments, we will use `n = 1000`. Again, we will still use $\epsilon$ = 0.4.

### Experiment 1: noise = 0.05

In [ ]:
np.random.seed(555)
# Generate the data set
X, y = datasets.make_moons(n_samples=n, shuffle=True, noise=0.05, random_state=None)

# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot11.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot11.png)

Spectral clustering has successfully found the two half-moon clusters.

### Experiment 2: noise = 0.1

In [ ]:
n = 1000
np.random.seed(555)
# Generate the data set
X, y = datasets.make_moons(n_samples=n, shuffle=True, noise=0.1, random_state=None)

# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot8.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot8.png)

Spectral clustering has successfully found the two half-moon clusters with only few misclassified points.

### Experiment 3: noise = 0.15

In [ ]:
np.random.seed(555)
# Generate the data set
X, y = datasets.make_moons(n_samples=n, shuffle=True, noise=0.15, random_state=None)

# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot9.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot9.png)

Spectral clustering has successfully found the two half-moon clusters, however there are quite a lot of misclassified points.

### Experiment 4: noise = 0.2

In [ ]:
np.random.seed(555)
# Generate the data set
X, y = datasets.make_moons(n_samples=n, shuffle=True, noise=0.2, random_state=None)

# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot10.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot10.png)

Spectral clustering can hardly found the two half-moon clusters. A lot of points have been misclassified. It is even worse than Experiment 4.

**From all our experiments, we can conclude as noise increases, the spectral clustering algorithm still works, but it starts to make lots of mistakes. I would not sugggest use spectral clustering when noise is large.**

## Part I

Now let's try our spectral clustering function on another data set -- the bull's eye! 

In [ ]:
n = 1000
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
plt.scatter(X[:,0], X[:,1])

![Blog-Post-4-Plot20.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot20.png)

There are two concentric circles. As before k-means will not do well here at all. 

In [ ]:
km = KMeans(n_clusters = 2)
km.fit(X)
plt.scatter(X[:,0], X[:,1], c = km.predict(X))

![Blog-Post-4-Plot12.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot12.png)

Now, let's try spectral clustering to see if it can successfully separate the two circles. We will test with a series of $\epsilon$ to get a range of $\epsilon$ that can correctly separate the two circles.

### Experiment 1: $\epsilon$ = 0.2

In [ ]:
epsilon = 0.2
n = 1000
# Generate the dataset
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot13.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot13.png)

When $\epsilon$ = 0.2, the spectral clustering algorithm cannot separate the two circles.

### Experiment 2: $\epsilon$ = 0.25

In [ ]:
epsilon = 0.25
n = 1000
# Generate the dataset
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot14.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot14.png)

When $\epsilon$= 0.25, the spectral clustering algorithm can successfully separate the two circles.

### Experiment 3: $\epsilon$ = 0.4

In [ ]:
epsilon = 0.4
n = 1000
# Generate the dataset
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot15.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot15.png)

When $\epsilon$= 0.4, the spectral clustering algorithm still can successfully separate the two circles.

### Experiment 4: $\epsilon$ = 0.5

In [ ]:
epsilon = 0.5
n = 1000
# Generate the dataset
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot16.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot16.png)

When $\epsilon$= 0.5, the spectral clustering algorithm still can successfully separate the two circles.

### Experiment 5: $\epsilon$ = 0.6

In [ ]:
epsilon = 0.6
n = 1000
# Generate the dataset
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot17.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot17.png)

When $\epsilon$= 0.6, the spectral clustering algorithm cannot separate the two circles.

### Experiment 6: $\epsilon$ = 0.52

In [ ]:
epsilon = 0.52
n = 1000
# Generate the dataset
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot18.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot18.png)

When $\epsilon$= 0.52, the spectral clustering algorithm still can successfully separate the two circles.

### Experiment 7: $\epsilon$ = 0.53

In [ ]:
epsilon = 0.53
n = 1000
# Generate the dataset
X, y = datasets.make_circles(n_samples=n, shuffle=True, noise=0.05, random_state=None, factor = 0.4)
# Perform spctral cluster
label = spectral_clustering(X, epsilon)
plt.scatter(X[:,0], X[:,1], c=label)

![Blog-Post-4-Plot19.png](https://raw.githubusercontent.com/JadenWSR/JadenWSR.github.io/master/images/Blog-Post-4-Plot19.png)

When $\epsilon$= 0.53, the spectral clustering algorithm cannot separate the two circles.

**After all the experiments, I have found that when 0.25$\leq \epsilon \leq$0.52, the spectral clustering algorithm can successfully separate the two circles.**